## Feed Forward Neural Network Models

### Import Libraries and Load Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string
import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist, bigrams
from wordcloud import WordCloud
from collections import Counter

In [2]:
df = pd.read_csv('tweets.csv')
df.head()

,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


### Data Preprocessing Functions

In [3]:
import re
import string
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import html

# Function to remove URLs
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

# Function to remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Function to remove HTML tags
def remove_html(text):
    html_tag = re.compile(r'<.*?>')
    return html_tag.sub(r'', text)

# Function to remove punctuation (including single and double quotes)
def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation + "“”’‘"))

# Function to remove HTML entities
def remove_html_entities(text):
    return html.unescape(text)

# Function to get POS tags in WordNet format
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# Applying preprocessing functions
df['text_clean'] = df['text'].apply(remove_URL)
df['text_clean'] = df['text_clean'].apply(remove_emoji)
df['text_clean'] = df['text_clean'].apply(remove_html)
df['text_clean'] = df['text_clean'].apply(remove_html_entities)
df['text_clean'] = df['text_clean'].apply(remove_punct)

# Tokenizing the clean text
df['tokenized'] = df['text_clean'].apply(word_tokenize)

# Lowercasing the tokenized text
df['lower'] = df['tokenized'].apply(lambda x: [word.lower() for word in x])

# Removing stopwords
stop = set(stopwords.words('english'))
df['stopwords_removed'] = df['lower'].apply(lambda x: [word for word in x if word not in stop])

# Applying part of speech tags
df['pos_tags'] = df['stopwords_removed'].apply(nltk.tag.pos_tag)

# Converting parts of speech to WordNet format
df['wordnet_pos'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

# Initialize the WordNet Lemmatizer
wnl = WordNetLemmatizer()

# Applying lemmatization using the WordNet POS tags
df['lemmatized'] = df['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x if tag is not None])

# Removing stopwords again from the lemmatized output
df['lemmatized'] = df['lemmatized'].apply(lambda x: [word for word in x if word not in stop])

# Concatenating the lemmatized words into a single string
df['final_text'] = df['lemmatized'].apply(' '.join)

df.head(10)

,id,keyword,location,text,target,text_clean,tokenized,lower,stopwords_removed,pos_tags,wordnet_pos,lemmatized,final_text
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1,Communal violence in Bhainsa Telangana Stones ...,"[Communal, violence, in, Bhainsa, Telangana, S...","[communal, violence, in, bhainsa, telangana, s...","[communal, violence, bhainsa, telangana, stone...","[(communal, JJ), (violence, NN), (bhainsa, NN)...","[(communal, a), (violence, n), (bhainsa, n), (...","[communal, violence, bhainsa, telangana, stone...",communal violence bhainsa telangana stone pelt...
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1,Telangana Section 144 has been imposed in Bhai...,"[Telangana, Section, 144, has, been, imposed, ...","[telangana, section, 144, has, been, imposed, ...","[telangana, section, 144, imposed, bhainsa, ja...","[(telangana, JJ), (section, NN), (144, CD), (i...","[(telangana, a), (section, n), (144, None), (i...","[telangana, section, impose, bhainsa, january,...",telangana section impose bhainsa january clash...
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1,Arsonist sets cars ablaze at dealership,"[Arsonist, sets, cars, ablaze, at, dealership]","[arsonist, sets, cars, ablaze, at, dealership]","[arsonist, sets, cars, ablaze, dealership]","[(arsonist, JJ), (sets, NNS), (cars, NNS), (ab...","[(arsonist, a), (sets, n), (cars, n), (ablaze,...","[arsonist, set, car, ablaze, dealership]",arsonist set car ablaze dealership
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1,Arsonist sets cars ablaze at dealership,"[Arsonist, sets, cars, ablaze, at, dealership]","[arsonist, sets, cars, ablaze, at, dealership]","[arsonist, sets, cars, ablaze, dealership]","[(arsonist, JJ), (sets, NNS), (cars, NNS), (ab...","[(arsonist, a), (sets, n), (cars, n), (ablaze,...","[arsonist, set, car, ablaze, dealership]",arsonist set car ablaze dealership
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0,Lord Jesus your love brings freedom and pardon...,"[Lord, Jesus, your, love, brings, freedom, and...","[lord, jesus, your, love, brings, freedom, and...","[lord, jesus, love, brings, freedom, pardon, f...","[(lord, NN), (jesus, NN), (love, VBP), (brings...","[(lord, n), (jesus, n), (love, v), (brings, n)...","[lord, jesus, love, brings, freedom, pardon, f...",lord jesus love brings freedom pardon fill hol...
5,5,ablaze,OC,"If this child was Chinese, this tweet would ha...",0,If this child was Chinese this tweet would hav...,"[If, this, child, was, Chinese, this, tweet, w...","[if, this, child, was, chinese, this, tweet, w...","[child, chinese, tweet, would, gone, viral, so...","[(child, NN), (chinese, JJ), (tweet, NN), (wou...","[(child, n), (chinese, a), (tweet, n), (would,...","[child, chinese, tweet, go, viral, social, med...",child chinese tweet go viral social medium abl...
6,6,ablaze,"London, England",Several houses have been set ablaze in Ngemsib...,1,Several houses have been set ablaze in Ngemsib...,"[Several, houses, have, been, set, ablaze, in,...","[several, houses, have, been, set, ablaze, in,...","[several, houses, set, ablaze, ngemsibaa, vill...","[(several, JJ), (houses, NNS), (set, VBD), (ab...","[(several, a), (houses, n), (set, v), (ablaze,...","[several, house, set, ablaze, ngemsibaa, villa...",several house set ablaze ngemsibaa village oku...
7,7,ablaze,Bharat,Asansol: A BJP office in Salanpur village was ...,1,Asansol A BJP office in Salanpur village was s...,"[Asansol, A, BJP, office, in, Salanpur, villag...","[asansol, a, bjp, office, in, salanpur, villag...","[asansol, bjp, office, salanpur, village, set,...","[(asansol, NNS), (bjp, JJ), (office, NN), (sal...","[(asansol, n), (bjp, a), (office, n), (salanpu...","[asansol, bjp, office, salanpur, village, set,...",asansol bjp office salanpur village set ablaze...
8,8,ablaze,"Accra, Ghana","National Security Minister, Kan Dapaah's side ...",0,National Secur

### Vectorisation (TF-IDF, CountVectorizer, Word2Vec, BERT)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch

In [5]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(df['final_text'], df['target'], test_size=0.2, random_state=42)

In [6]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [7]:
# CountVectorizer
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)


In [8]:
# Word2Vec Vectorization
sentences = [text.split() for text in df['final_text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.train(sentences, total_examples=len(sentences), epochs=10)

def get_word2vec_embeddings(text, model, vector_size):
    words = text.split()
    embedding = np.zeros(vector_size)
    count = 0
    for word in words:
        if word in model.wv.key_to_index:
            embedding += model.wv[word]
            count += 1
    if count > 0:
        embedding /= count
    return embedding

X_train_word2vec = np.array([get_word2vec_embeddings(text, word2vec_model, 100) for text in X_train])
X_test_word2vec = np.array([get_word2vec_embeddings(text, word2vec_model, 100) for text in X_test])


In [9]:
# BERT Embeddings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

X_train_bert = np.array([get_bert_embeddings(text, tokenizer, model) for text in X_train])
X_test_bert = np.array([get_bert_embeddings(text, tokenizer, model) for text in X_test])


In [10]:
# Ensure you have the correct shapes
print(X_train_tfidf.shape, X_train_count.shape, X_train_word2vec.shape, X_train_bert.shape)

(9096, 16569) (9096, 16569) (9096, 100) (9096, 768)


### Addressing Class Imbalance

In [11]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Ensure y_train and y_test are numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Check unique values in y_train
unique_classes = np.unique(y_train)
print("Unique classes in y_train:", unique_classes)

# Compute class weights correctly
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train)
class_weights_dict = dict(zip(unique_classes, class_weights))
print("Updated class weights dict:", class_weights_dict)

Unique classes in y_train: [0 1]
Updated class weights dict: {0: 0.6164272160477094, 1: 2.647264260768335}


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Metric

### Feed forward Neural Network(FFNN) Model Building and Training

In [13]:
class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)
    
    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    
    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [14]:
def create_ffnn(input_shape):
    model = Sequential()
    model.add(Dense(128, input_shape=(input_shape,), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # For binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[F1Score()])
    return model

In [15]:
# Using TF-IDF vectorized data
input_shape = X_train_tfidf.shape[1]
model_tfidf = create_ffnn(input_shape)

# Fit the model with class weights
model_tfidf.fit(X_train_tfidf.toarray(), y_train, epochs=10, batch_size=32, validation_data=(X_test_tfidf.toarray(), y_test), class_weight=class_weights_dict, callbacks=[EarlyStopping(patience=3)])

C:\Users\Aravinthkumar\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - f1_score: 0.4436 - loss: 0.6094 - val_f1_score: 0.6243 - val_loss: 0.3980
Epoch 2/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - f1_score: 0.8444 - loss: 0.1925 - val_f1_score: 0.6393 - val_loss: 0.3653
Epoch 3/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - f1_score: 0.9452 - loss: 0.0666 - val_f1_score: 0.6642 - val_loss: 0.3828
Epoch 4/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - f1_score: 0.9853 - loss: 0.0222 - val_f1_score: 0.6474 - val_loss: 0.4326
Epoch 5/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - f1_score: 0.9883 - loss: 0.0152 - val_f1_score: 0.6634 - val_loss: 0.4688


In [16]:
# Using CountVectorizer data
input_shape = X_train_count.shape[1]
model_count = create_ffnn(input_shape)

# Fit the model with class weights
model_count.fit(X_train_count.toarray(), y_train, epochs=10, batch_size=32, validation_data=(X_test_count.toarray(), y_test), class_weight=class_weights_dict, callbacks=[EarlyStopping(patience=3)])

Epoch 1/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - f1_score: 0.4741 - loss: 0.5836 - val_f1_score: 0.6177 - val_loss: 0.4347
Epoch 2/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - f1_score: 0.8453 - loss: 0.1826 - val_f1_score: 0.6917 - val_loss: 0.3159
Epoch 3/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - f1_score: 0.9655 - loss: 0.0460 - val_f1_score: 0.6810 - val_loss: 0.3738
Epoch 4/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - f1_score: 0.9850 - loss: 0.0201 - val_f1_score: 0.7051 - val_loss: 0.4249
Epoch 5/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - f1_score: 0.9950 - loss: 0.0088 - val_f1_score: 0.7000 - val_loss: 0.4604


In [17]:
# Using Word2Vec embeddings
input_shape = X_train_word2vec.shape[1]
model_word2vec = create_ffnn(input_shape)

# Fit the model with class weights
model_word2vec.fit(X_train_word2vec, y_train, epochs=10, batch_size=32, validation_data=(X_test_word2vec, y_test), class_weight=class_weights_dict, callbacks=[EarlyStopping(patience=3)])

Epoch 1/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - f1_score: 0.4526 - loss: 0.5953 - val_f1_score: 0.5349 - val_loss: 0.4682
Epoch 2/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - f1_score: 0.5230 - loss: 0.5516 - val_f1_score: 0.5050 - val_loss: 0.5287
Epoch 3/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - f1_score: 0.5325 - loss: 0.5333 - val_f1_score: 0.5355 - val_loss: 0.4395
Epoch 4/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - f1_score: 0.5313 - loss: 0.5263 - val_f1_score: 0.5383 - val_loss: 0.4593
Epoch 5/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - f1_score: 0.5154 - loss: 0.5453 - val_f1_score: 0.5025 - val_loss: 0.5322
Epoch 6/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - f1_score: 0.5321 - loss: 0.5341 - val_f1_score: 0.5204 - val_loss: 0.5122


In [18]:
# Using BERT embeddings
input_shape = X_train_bert.shape[1]
model_bert = create_ffnn(input_shape)

# Fit the model with class weights
model_bert.fit(X_train_bert, y_train, epochs=10, batch_size=32, validation_data=(X_test_bert, y_test), class_weight=class_weights_dict, callbacks=[EarlyStopping(patience=3)])

Epoch 1/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - f1_score: 0.5486 - loss: 0.4962 - val_f1_score: 0.6006 - val_loss: 0.4273
Epoch 2/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.6299 - loss: 0.4027 - val_f1_score: 0.5973 - val_loss: 0.4311
Epoch 3/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - f1_score: 0.6498 - loss: 0.3742 - val_f1_score: 0.5810 - val_loss: 0.4909
Epoch 4/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - f1_score: 0.6714 - loss: 0.3391 - val_f1_score: 0.6086 - val_loss: 0.4197
Epoch 5/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.7021 - loss: 0.3057 - val_f1_score: 0.6131 - val_loss: 0.3995
Epoch 6/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.7520 - loss: 0.2538 - val_f1_score: 0.6206 - val_loss: 0.4015
Epoch 7/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.7901 - loss: 0.2214 - val_f1_score: 0.6320 - val_loss: 0.3769
Epoch 8/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.8131 - loss: 0.1844 - val_f1_score: 0.

### Evaluate Models and Compile Metrics

In [21]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

# Function to calculate evaluation metrics
def evaluate_model(model, X_test, y_test):
    y_pred_prob = model.predict(X_test).ravel()
    y_pred = (y_pred_prob > 0.5).astype(int)
    
    auc = roc_auc_score(y_test, y_pred_prob)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    return {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    }


In [22]:
# Initialize a dictionary to store the metrics for each model
metrics = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1-Score': []
}

# Evaluate the TF-IDF model
tfidf_metrics = evaluate_model(model_tfidf, X_test_tfidf.toarray(), y_test)
metrics['Model'].append('TF-IDF')
for metric, value in tfidf_metrics.items():
    metrics[metric].append(value)

# Evaluate the CountVectorizer model
count_metrics = evaluate_model(model_count, X_test_count.toarray(), y_test)
metrics['Model'].append('CountVectorizer')
for metric, value in count_metrics.items():
    metrics[metric].append(value)

# Evaluate the Word2Vec model
word2vec_metrics = evaluate_model(model_word2vec, X_test_word2vec, y_test)
metrics['Model'].append('Word2Vec')
for metric, value in word2vec_metrics.items():
    metrics[metric].append(value)

# Evaluate the BERT model
bert_metrics = evaluate_model(model_bert, X_test_bert, y_test)
metrics['Model'].append('BERT')
for metric, value in bert_metrics.items():
    metrics[metric].append(value)

# Create a DataFrame from the metrics dictionary
metrics_df = pd.DataFrame(metrics)

72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


## FFNN Metrics

In [23]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score
0,TF-IDF,0.878628,0.641509,0.686869,0.663415
1,CountVectorizer,0.905013,0.777778,0.636364,0.700000
2,Word2Vec,0.772208,0.410819,0.709596,0.520370
3,BERT,0.856201,0.566219,0.744949,0.643402
